In [5]:
import cv2
import os
from os.path import join, basename
import ntpath
import json

In [6]:
base_dir = 'orig'
output_dir = 'canny'

# coding=utf-8
import cv2
import numpy as np
import os
from os.path import join, basename

def compute_slope(line):
    x1, y1, x2, y2 = line[0]
    return (y2 - y1) / (x2 - x1 + np.finfo(float).eps)

def compute_slope_tan(line):
    x1, y1, x2, y2 = line[0]
    return np.rad2deg(np.arctan2(y2 - y1, x2 - x1 + np.finfo(float).eps))

def compute_bias(line):
    x1, y1, x2, y2 = line[0]
    return y1 - compute_slope(line) * x1

def compute_intercept(line):
    x1, y1, x2, y2 = line[0]
    return (y2 - y1) / (x2 - x1 + np.finfo(float).eps)

def yintercept(line):
    x1, y1, x2, y2 = line
    '''Get the y intercept of a line segment'''
    # if compute_slope(line) != None:
    x, y = x1,y1
    return y - compute_slope1(line) * x
    # else:
    #     return None

def compute_slope1(line):
    x1, y1, x2, y2 = line
    return (y2 - y1) / (x2 - x1 + np.finfo(float).eps)

def compute_lane_from_candidates(line_candidates, img_shape):
        """
        Compute lines that approximate the position of both road lanes.

        :param line_candidates: lines from hough transform
        :param img_shape: shape of image to which hough transform was applied
        :return: lines that approximate left and right lane position
        """

        pos_lines = [l for l in line_candidates if compute_slope(l) > 0]
        neg_lines = [l for l in line_candidates if compute_slope(l) < 0]

        # interpolate biases and slopes to compute equation of line that approximates left lane
        # median is employed to filter outliers
        neg_bias = np.median([compute_bias(l) for l in neg_lines]).astype(int)
        neg_slope = np.median([compute_slope(l) for l in neg_lines])
#         print("neg_slope", neg_slope)
#         print("neg_bias", neg_bias)
        x1, y1 = 0, neg_bias
        x2, y2 = -np.int32(np.round(neg_bias / neg_slope)), 0
        left_lane = x1, y1, x2, y2

        # interpolate biases and slopes to compute equation of line that approximates right lane
        # median is employed to filter outliers

        lane_right_bias = np.median([compute_bias(l) for l in pos_lines]).astype(int)
        lane_right_slope = np.median([compute_slope(l) for l in pos_lines])
#         print("lane_right_bias", lane_right_bias)
#         print("lane_right_slope", lane_right_slope)
        x1, y1 = 0, lane_right_bias
        x2, y2 = np.int32(np.round((img_shape[0] - lane_right_bias) / lane_right_slope)), img_shape[0]
        right_lane = x1, y1, x2, y2
        
#         if yintercept(left_lane) < 0.001:
#             yintercept(left_lane) = 0.001
            
#         if yintercept(right_lane) < 0.001:
#             yintercept(right_lane) = 0.001

#         print("lane_right_intercept", yintercept(left_lane))
#         print("lane_right_intercept", yintercept(right_lane))

        return left_lane, right_lane

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

images_dirs = [join(base_dir, name) for name in os.listdir(base_dir)]
for folder in images_dirs: 
     if ntpath.basename(folder).startswith('tub'):        
        des = output_dir + '/' + ntpath.basename(folder) + '_f'
#         print(des)
        if not os.path.exists(output_dir + '/' + ntpath.basename(folder) + '_f'):
            os.mkdir(output_dir + '/' + ntpath.basename(folder) + '_f')

        test_images = [join(folder, name) for name in os.listdir(folder)]
        for img in test_images:
            if img.endswith('.jpg'):
#                 image = cv2.imread(img)

                img_orig = cv2.imread(img)
                image = cv2.imread(img, 0)
#                 image = image[0:60, 160:60] 

                # img = cv2.GaussianBlur(img, (3,3),0)
                edges = cv2.Canny(image, 40, 70)
                height, width = image.shape

                img2 = np.ones((height, width, 1), np.uint8) * 255

                detected_lines = cv2.HoughLinesP(edges, 2, np.pi / 180, 1, minLineLength=15, maxLineGap=5)

                candidate_lines = []
                for line in detected_lines:
                    # consider only lines with slope between 30 and 60 degrees
                    # x1, y1, x2, y2 = line[0]
                    if 0.5 <= np.abs(compute_slope(line)) <= 2:
                        # print(line)
                        # print(yintercept(line))
                        candidate_lines.append(line)

                # left_lane, right_lane = compute_lane_from_candidates(detected_lines,img.shape)
                left_lane, right_lane = compute_lane_from_candidates(candidate_lines, image.shape)

                for line in candidate_lines:
                    x1, y1, x2, y2 = line[0]
                    if y1 > 60:
                        cv2.line(img_orig, (x1, y1), (x2, y2), (0, 0, 255), 3)
                    # cv2.line(img2, (x1, y1), (x2, y2), (0, 255, 0), 3)

                # cv2.imshow("Edges", edges)
                # cv2.imshow("Orig", img)

#                 x1, y1, x2, y2 = left_lane
#                 try:
#                     cv2.line(img_orig, (x1, y1), (x2, y2), (0, 0, 0), 3)
#                 except OverflowError:
#                     print ("Unexpected error:")
#                     raise    
# #                 cv2.line(img2, (x1, y1), (x2, y2), (0, 255, 0), 3)

#                 x1, y1, x2, y2 = right_lane
#                 try:
#                     cv2.line(img_orig, (x1, y1), (x2, y2), (0, 0, 0), 3)
#                 except OverflowError :
#                     print ("Unexpected error:")
#                     raise 
#                 cv2.line(img2, (x1, y1), (x2, y2), (0, 255, 0), 3)

                cv2.imwrite(des + '/' + ntpath.basename(img) , img_orig)

#             if img.endswith('.json') & ntpath.basename(img).startswith('record'):
#                 with open(img) as f:
#                     data = json.load(f)

#                     data1 = {}  
#                     data1['cam/image_array'] = data['cam/image_array']
#                     data1['user/mode'] = data['user/mode']
#                     data1['timestamp'] = data['timestamp']
#                     data1['user/throttle'] = data['user/throttle']
#                     data1['user/angle'] = data['user/angle'] * -1

#                     with open(des + '/' + ntpath.basename(img), 'w') as outfile:  
#                         json.dump(data1, outfile)

            if img.endswith('.json'):
                with open(img) as f:
                    data = json.load(f)

                    with open(des + '/' + ntpath.basename(img), 'w') as outfile:  
                        json.dump(data, outfile)

lane_right_intercept 54.0
lane_right_intercept -133.0
lane_right_intercept 118.0
lane_right_intercept -9.22337203685e+18
lane_right_intercept 101.0
lane_right_intercept -57.0
lane_right_intercept 100.0
lane_right_intercept -59.0
lane_right_intercept 78.0
lane_right_intercept 43.0
lane_right_intercept 120.0
lane_right_intercept 6.0
lane_right_intercept 123.0
lane_right_intercept -106.0
lane_right_intercept 119.0
lane_right_intercept -67.0
lane_right_intercept 140.0
lane_right_intercept -38.0
lane_right_intercept 116.0
lane_right_intercept -27.0
lane_right_intercept 106.0
lane_right_intercept -53.0
lane_right_intercept 124.0
lane_right_intercept -9.22337203685e+18
lane_right_intercept 189.0
lane_right_intercept -139.0
lane_right_intercept 113.0
lane_right_intercept -9.22337203685e+18


/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:79: RuntimeWarning: overflow encountered in long_scalars
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: overflow encountered in long_scalars


lane_right_intercept 128.0
lane_right_intercept 8.0
lane_right_intercept 99.0
lane_right_intercept -50.0
lane_right_intercept 115.0
lane_right_intercept -102.0
lane_right_intercept 131.0
lane_right_intercept -9.22337203685e+18
lane_right_intercept 92.0
lane_right_intercept -59.0
lane_right_intercept 98.0
lane_right_intercept -9.22337203685e+18
lane_right_intercept 102.0
lane_right_intercept -115.0
lane_right_intercept 109.0
lane_right_intercept 1.0
lane_right_intercept 94.0
lane_right_intercept -9.22337203685e+18
lane_right_intercept 63.0
lane_right_intercept -41.0
lane_right_intercept 118.0
lane_right_intercept -32.0
lane_right_intercept 134.0
lane_right_intercept -9.22337203685e+18
lane_right_intercept 91.0
lane_right_intercept -78.0
lane_right_intercept 100.0
lane_right_intercept 4.0
lane_right_intercept 127.0
lane_right_intercept -54.0
lane_right_intercept 101.0
lane_right_intercept -72.0
lane_right_intercept -9.22337203685e+18
lane_right_intercept 20.0
lane_right_intercept 90.0
la